In [1]:
import wfdb
import csv
import pandas as pd
import numpy as np
import os

In [2]:
import os
import csv
import pandas as pd
import wfdb  # Make sure wfdb is installed

output_dir = "data_creation"
os.makedirs(output_dir, exist_ok=True)

# Patient numbers
patient_numbers = [
    "100", "101", "102", "103", "104", "105", "106", "107", "108", "109",
    "111", "112", "113", "114", "115", "116", "117", "118", "119", "121",
    "122", "123", "124", "200", "201", "202", "203", "205", "207", "208",
    "209", "210", "212", "213", "214", "215", "217", "219", "220", "221",
    "222", "223", "228", "230", "231", "232", "233", "234"
]

# N = normal 
# S = supra-ventricular premature
# V = ventricular escape
# F = fusion of ventricular and normal
# Q = unclassified heartbeats
symbol_to_category = {
    'N': 'N', '.': 'N', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N',
    'a': 'S', 'A': 'S', 'J': 'S', 'S': 'S',
    'V': 'V', 'E': 'V',
    'F': 'F',
    '/': 'Q', 'f': 'Q', 'Q': 'Q'
}

for patient_number in patient_numbers:
    try:
        # ECG data
        path_to_record = f"mit-database/{patient_number}"
        patient_record = wfdb.rdrecord(path_to_record)
        leads = patient_record.sig_name
        ecg_data = patient_record.p_signal

        # ECG CSV
        ecg_filename = f"{output_dir}/{patient_number}_ECG.csv"
        with open(ecg_filename, "w", newline='') as outfile:
            out_csv = csv.writer(outfile)
            out_csv.writerow(leads)
            for row in ecg_data:
                out_csv.writerow(row)

        # Annotations data
        annotation = wfdb.rdann(path_to_record, 'atr')
        symbols = annotation.symbol
        annotations = annotation.sample

        # Filter out symbols not in symbol_to_category
        filtered_symbols_annotations = [(sym, ann) for sym, ann in zip(symbols, annotations) if sym in symbol_to_category]
        categories = [symbol_to_category[sym] for sym, ann in filtered_symbols_annotations]
        annotations_filtered = [ann for sym, ann in filtered_symbols_annotations]

        df_annotations = pd.DataFrame({'Category': categories, 'Annotation': annotations_filtered})

        # Annotations CSV
        annotations_filename = f"{output_dir}/{patient_number}_Annotations.csv"
        df_annotations.to_csv(annotations_filename, index=False)

    except Exception as e:
        print(f"Failed to process: {patient_number}: {e}")

print("Done")

Done


In [3]:
import os
import pandas as pd
import numpy as np


def process_patient_data(patient_number, data_creation_dir="data_creation"):
    ecg_file_path = os.path.join(data_creation_dir, f"{patient_number}_ECG.csv")
    annotations_file_path = os.path.join(data_creation_dir, f"{patient_number}_Annotations.csv")
    
    patient_X = []
    patient_Y = []
    
    try:
        ecg_df = pd.read_csv(ecg_file_path)
        annotations_df = pd.read_csv(annotations_file_path)
    except FileNotFoundError:
        print(f"Files for patient {patient_number} not found. Skipping...")
        return [], []
    
    first_column_name = ecg_df.columns[0]
    second_column_name = ecg_df.columns[1] if len(ecg_df.columns) > 1 else None

    sampling_rate = 360  # Hz
    total_window_size_seconds = 1  # Total window size in seconds
    total_window_size_samples = total_window_size_seconds * sampling_rate

    for _, row in annotations_df.iterrows():
        annotation_point = row['Annotation']
        category = row['Category']
        
        # Randomly determine the window split around the annotation point
        before_seconds = np.random.uniform(0, total_window_size_seconds)
        after_seconds = total_window_size_seconds - before_seconds
        before_samples = int(before_seconds * sampling_rate)
        after_samples = int(after_seconds * sampling_rate)
        
        start_point = max(0, annotation_point - before_samples)
        end_point = start_point + total_window_size_samples  # Ensure the window is of the exact expected size
        
        # Adjust the end point if it exceeds the length of the data
        if end_point > len(ecg_df):
            end_point = len(ecg_df)
            start_point = max(0, end_point - total_window_size_samples)  # Adjust start point accordingly
        
        # Process data from the first column
        window_data_first_column = ecg_df.iloc[start_point:end_point][first_column_name].to_numpy()
        
        patient_X.append(window_data_first_column)
        patient_Y.append(category)
        
        # If there's a second column, process it and add as a new entry
        if second_column_name:
            window_data_second_column = ecg_df.iloc[start_point:end_point][second_column_name].to_numpy()
            
            patient_X.append(window_data_second_column)
            patient_Y.append(category)  # Repeat category for the new entry
    
    return patient_X, patient_Y



# Initialize lists to hold the entire dataset
all_X = []
all_Y = []

data_creation_dir = "data_creation"

# Process each patient
for patient_number in patient_numbers:
    patient_X, patient_Y = process_patient_data(patient_number, data_creation_dir)
    all_X.extend(patient_X)
    all_Y.extend(patient_Y)

X = np.array(all_X)
Y = np.array(all_Y)

In [4]:
X

array([[ 0.375,  0.62 ,  0.78 , ..., -0.405, -0.395, -0.39 ],
       [ 0.535,  0.58 ,  0.475, ..., -0.28 , -0.275, -0.275],
       [-0.36 , -0.38 , -0.425, ..., -0.4  , -0.395, -0.415],
       ...,
       [-0.005,  0.005, -0.015, ...,  0.075,  0.075,  0.085],
       [-0.25 , -0.24 , -0.22 , ..., -0.405, -0.405, -0.4  ],
       [ 0.035,  0.03 ,  0.05 , ...,  0.005,  0.01 ,  0.025]])

In [5]:
Y

array(['N', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U1')

In [6]:
df_x = pd.DataFrame(X)
df_x

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,0.375,0.620,0.780,0.840,0.765,0.520,0.170,-0.165,-0.365,-0.435,...,-0.405,-0.410,-0.390,-0.390,-0.385,-0.385,-0.410,-0.405,-0.395,-0.390
1,0.535,0.580,0.475,0.210,-0.085,-0.230,-0.250,-0.215,-0.180,-0.185,...,-0.275,-0.280,-0.270,-0.265,-0.275,-0.285,-0.280,-0.280,-0.275,-0.275
2,-0.360,-0.380,-0.425,-0.445,-0.475,-0.510,-0.535,-0.505,-0.415,-0.300,...,-0.410,-0.410,-0.425,-0.420,-0.400,-0.395,-0.400,-0.400,-0.395,-0.415
3,-0.290,-0.300,-0.330,-0.360,-0.355,-0.305,-0.205,-0.080,0.015,0.065,...,-0.255,-0.260,-0.275,-0.270,-0.275,-0.265,-0.265,-0.270,-0.280,-0.285
4,-0.335,-0.340,-0.335,-0.335,-0.330,-0.325,-0.345,-0.350,-0.345,-0.335,...,-0.265,-0.255,-0.245,-0.260,-0.265,-0.275,-0.280,-0.265,-0.270,-0.275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218983,0.175,0.175,0.150,0.150,0.150,0.135,0.140,0.150,0.155,0.175,...,-0.020,0.015,0.010,0.025,0.010,-0.005,0.000,-0.015,-0.040,-0.050
218984,-0.280,-0.290,-0.295,-0.280,-0.235,-0.135,0.010,0.210,0.460,0.735,...,-0.280,-0.270,-0.270,-0.275,-0.275,-0.265,-0.250,-0.240,-0.235,-0.265
218985,-0.005,0.005,-0.015,-0.005,-0.005,0.005,0.010,-0.005,0.015,0.055,...,-0.005,0.000,0.000,0.020,0.025,0.040,0.070,0.075,0.075,0.085
218986,-0.250,-0.240,-0.220,-0.210,-0.225,-0.225,-0.240,-0.240,-0.240,-0.240,...,-0.275,-0.280,-0.285,-0.305,-0.330,-0.365,-0.390,-0.405,-0.405,-0.400


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_x = scaler.fit_transform(df_x)
df_x = pd.DataFrame(df_x)

In [8]:
import pywt

def madev(d, axis=None):
    'Mean absolute deviation of a signal'
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def wavelet_denoising(x, wavelet='sym4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')


df_x = wavelet_denoising(df_x, wavelet='sym4', level=2)
df_x = pd.DataFrame(df_x)

In [9]:
df_y = pd.DataFrame(Y, columns=['ColumnName'])
print(df_y)

       ColumnName
0               N
1               N
2               N
3               N
4               N
...           ...
218983          N
218984          N
218985          N
218986          N
218987          N

[218988 rows x 1 columns]


In [10]:
value_counts_y = df_y['ColumnName'].value_counts()
print(value_counts_y)

N    181262
Q     16086
V     14472
S      5562
F      1606
Name: ColumnName, dtype: int64


In [11]:
df_fusionné = pd.concat([df_y, df_x], axis=1)
df_fusionné

,ColumnName,0,1,2,3,4,5,6,7,8,...,350,351,352,353,354,355,356,357,358,359
0,N,0.535963,0.558094,0.579907,0.584880,0.572034,0.549807,0.514988,0.477454,0.466336,...,0.461713,0.465465,0.463000,0.458537,0.460518,0.465057,0.460692,0.460204,0.463192,0.461308
1,N,0.551337,0.556537,0.546606,0.516833,0.496502,0.477127,0.478523,0.486988,0.484651,...,0.474366,0.476848,0.474384,0.470380,0.471829,0.475804,0.471864,0.471835,0.476839,0.471536
2,N,0.460577,0.459287,0.460181,0.461207,0.457592,0.453899,0.448771,0.445308,0.455312,...,0.459965,0.459971,0.459472,0.458913,0.457981,0.457110,0.457666,0.458631,0.459785,0.461001
3,N,0.473704,0.470662,0.466724,0.463517,0.467258,0.474042,0.481339,0.489463,0.498357,...,0.476593,0.477735,0.477047,0.475781,0.473765,0.471726,0.471679,0.472072,0.473975,0.475977
4,N,0.472152,0.471809,0.471441,0.471033,0.470583,0.470121,0.469617,0.469121,0.468766,...,0.471993,0.472229,0.472369,0.472457,0.472482,0.472466,0.472487,0.472483,0.472466,0.472401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218983,N,0.504002,0.505105,0.506176,0.507272,0.508490,0.509686,0.510940,0.512096,0.512699,...,0.499138,0.498610,0.498701,0.499083,0.499641,0.500340,0.500933,0.501563,0.502254,0.503015
218984,N,0.477601,0.474407,0.471439,0.470020,0.477608,0.490557,0.501816,0.515232,0.541793,...,0.478491,0.481178,0.479877,0.477347,0.473545,0.469781,0.470008,0.471374,0.475293,0.479740
218985,N,0.502679,0.502859,0.503942,0.505072,0.504025,0.502766,0.501117,0.500006,0.503852,...,0.498768,0.499275,0.499526,0.499715,0.499801,0.499892,0.500396,0.501036,0.501638,0.502279
218986,N,0.471753,0.472184,0.472545,0.472847,0.473648,0.474588,0.475551,0.476493,0.477128,...,0.466174,0.472607,0.472019,0.469484,0.466916,0.464395,0.464917,0.466519,0.468592,0.470880


In [12]:
df_fusionné.to_csv('df_fusionné.csv', index=False)